In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
torch.manual_seed(1) # 똑같은 결과를 위한 random_seed 고정


# Training with Low-Level Cross Entropy Loss


In [7]:
X_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)

In [25]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

epochs=1000
for epoch in range(epochs+1):
    
    # Cost 계산(1)
    hypothesis = F.softmax(X_train.matmul(W) + b, dim=1) 
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(X) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100 ==0:
        print('Epoch {:4d}/ {} Cost : {:.6f}'.format(epoch, epochs, cost.item()))

Epoch    0/ 1000 Cost : 1.098612
Epoch  100/ 1000 Cost : 0.901535
Epoch  200/ 1000 Cost : 0.839114
Epoch  300/ 1000 Cost : 0.807826
Epoch  400/ 1000 Cost : 0.788472
Epoch  500/ 1000 Cost : 0.774822
Epoch  600/ 1000 Cost : 0.764449
Epoch  700/ 1000 Cost : 0.756191
Epoch  800/ 1000 Cost : 0.749398
Epoch  900/ 1000 Cost : 0.743671
Epoch 1000/ 1000 Cost : 0.738749


# Training with F.corss_entropy


In [26]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad = True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

epochs=1000

for epoch in range(epochs + 1):
    # Cost 계산(2)
    z = X_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    # cost로 H(X) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


# High-level Implementation with nn.Module


In [37]:
class SoftmaxClassifiermodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Ouput이 3
    def forward(self, x):
        return self.linear(x)

In [38]:
model = SoftmaxClassifiermodel()

In [39]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

epochs=1000
for epoch in range(epochs+1):
    
    # H(x) 계산
    prediction = model(X_train)
    
    # Cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))
        

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428280
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818
